In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.neural_network import MLPRegressor

# Load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Add the parent directory to the Python path
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_fit_tuning, fun_load_best_params
from ML_functions import fun_convert_time
from ML_functions import fun_tuning_results, fun_scores

# Start time count
start_script = time.time()

# Assign string 'TSP' or 'CVRP' to the following variable to define the optimization problem
optimization_problem = 'Bin_Packing'
train_size = 0.75

# Load data
data = fun_load_data(optimization_problem=optimization_problem)
X, y, train_data = fun_preprocessing(data)

# Create a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)

# Save train size and create a dictionary to store the results
train_size = f'{int(np.round(100 * len(X_train)/len(X)))} %'
results_dict = {}

# **Neural Network - Multi Layer Perceptron**
**Hyperparametertuning**

In [2]:
# Define a pipeline
pipe = make_pipeline(StandardScaler(),
                     MLPRegressor(activation='relu', solver='sgd', learning_rate='adaptive', batch_size='auto', 
                                  max_iter=10000, shuffle=True, verbose=False, random_state=42))

# Define parameter grid
param_grid = {'mlpregressor__hidden_layer_sizes': [(100, 100), (50, 50, 50), (100, 100, 100)],
              'mlpregressor__alpha': [0.05, 0.1, 0.5, 1, 2]}

# Grid search
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, 
                           scoring='neg_mean_absolute_percentage_error', verbose=True, n_jobs=-1)
fit_time = fun_fit_tuning(search_method=grid_search, X_train=X_train, y_train=y_train, file_name=optimization_problem + '_NN_GS_best_params.pkl')

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=grid_search, X_train=X_train, y_train=y_train, cv=3)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(search_method=grid_search, search_space=param_grid)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
CV MAPE train data:  13.3951 %
CV RMSE train data:  0.0501

Best model / parameter combination:


{'mlpregressor__alpha': 1, 'mlpregressor__hidden_layer_sizes': (100, 100)}

Cross validation scores of different parameter combinations:


,param_mlpregressor__hidden_layer_sizes,param_mlpregressor__alpha,mean_test_score,converted_mean_fit_time
0,"(100, 100)",1,-0.133951,19s
1,"(50, 50, 50)",1,-0.147015,13s
2,"(100, 100, 100)",0.5,-0.153001,22s
3,"(100, 100, 100)",1,-0.160263,31s
4,"(100, 100)",2,-0.173598,16s
5,"(50, 50, 50)",2,-0.224846,13s
6,"(100, 100)",0.5,-0.240211,6s
7,"(50, 50, 50)",0.5,-0.246368,3s
8,"(100, 100, 100)",2,-0.246729,18s
9,"(100, 100, 100)",0.1,-0.382491,1s


**Test Score**

In [3]:
# Load best parameters of the model
best_params = fun_load_best_params(optimization_problem + '_NN_GS_best_params.pkl')

# Create pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(solver='sgd', activation='relu', learning_rate='adaptive', 
                                  batch_size='auto', max_iter=10000, shuffle=True, verbose=False, random_state=0))
pipe.set_params(**best_params)

# Estimate model performance with cross-validation on the train set and get scores on test set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=pipe, X_train=X_train, y_train=y_train, cv=3, X_test=X_test, y_test=y_test, train_data=train_data)
results_dict['Neural Network'] = model_results_dict

{'mlpregressor__alpha': 1, 'mlpregressor__hidden_layer_sizes': (100, 100)}

CV MAPE train data:  13.5183 %
CV RMSE train data:  0.0632
CV computation time: 6s

MAPE test data: 14.838000000000001 %
RMSE test data: 0.0683

MAPE and RMSE on test data per instance size:


Number Items,5,6,7,8,9,10,11,12,13,14,Mean
MAPE,10.7341,10.0426,21.0292,6.0424,11.5001,3.1254,7.5208,10.9966,16.0593,25.0292,14.8380
RMSE,0.0756,0.0605,0.0968,0.0405,0.0431,0.0161,0.0395,0.0466,0.0660,0.0863,0.0683
